### Notes On Sat_Dat Tensorbuilding

- Two different possible storage formats
    - Var > Time > Lat > Long > Data Point
        - Easier to build
        - Likely larger size
        - Probably faster runtime
    - Time > Lat > Long > [Var1, Var2, Var3]
        - Harder to build
        - Will require organizing vars in memory for efficiency
            - Make data structure, put parts in bit by bit
        - Will likely take much longer to run
        

- Multithreading would be great
    - Set queue of all variables to process
    - Grab oldest var from queue
        - process data
        - put into individual var_named file
        - If failure, put variable back into queue
        
        
        
- GFS is a forecast, I don't want to be forecasting a forecast probably.  Could still be useful as a back up or for validation, but now I'm looking for non-forecast waveheight measurements
- Running into issues with ftp servers, sticking with GFS predictions.  Model is now more of a translator than predictor
- How to deal with updates, get new data daily and overwrite existing data?

In [1]:
import requests, json, sys, time, datetime, urllib, ftplib, os
import pandas as pd

import xarray as xr

import numpy as np
from netCDF4 import Dataset
url='http://nomads.ncep.noaa.gov:80/dods/wave/gfswave/20220305/gfswave.epacif.0p16_00z'
url='http://nomads.ncep.noaa.gov:80/dods/wave/gfswave/20220305/gfswave.wcoast.0p16_00z'
url='http://nomads.ncep.noaa.gov:80/dods/wave/gfswave/20220305/gfswave.gsouth.0p25_12z' # This one has data [0,0]
url='http://nomads.ncep.noaa.gov:80/dods/wave/gfswave/20220305/gfswave.gsouth.0p25_18z' # This one has data [100,100]

In [3]:
# test = ftplib.ftp_get(host, product_id, dataset_id, user, password)
# https://sealevel.jpl.nasa.gov/data/get-data/ sketchy data 

# # da = xr.open_dataset(url)
# # da['dirpwsfc']
# host = "nrt.cmems-du.eu"
# user = "rwitt"
# password = "guGpa7-myzfug-syvhus"
# product_id = "WAVE_GLO_WAV_L3_SPC_NRT_OBSERVATIONS_014_002"
# dataset_id = 'cmems_obs-wave_glo_phy-spc_nrt_cfo-l3_PT10S'
# file = 'ftp://nrt.cmems-du.eu/Core/WAVE_GLO_WAV_L3_SPC_NRT_OBSERVATIONS_014_002/cmems_obs-wave_glo_phy-spc_nrt_cfo-l3_PT10S'


# with ftplib.FTP(host) as ftp:
#     ftp.login(user, password)
#     ftp.dir()
#     ftp.cwd('/Core')
#     ftp.cwd('WAVE_GLO_WAV_L3_SWH_NRT_OBSERVATIONS_014_001')
#     ftp.dir()
#     ftp.cwd('dataset-wav-alti-l3-swh-rt-global-al')
#     ftp.dir()
#     ftp.cwd('2022')
#     ftp.dir()
#     ftp.cwd('01')
#     ftp.dir()
# #     with open('/Users/rowenwitt/downloads/test', 'wb') as fp:
            
# #             res = ftp.retrbinary('RETR ' + 'global_vavh_l3_rt_al_20220131T180000_20220131T210000_20220201T000134.nc', fp.write)
    
# with open('/Users/rowenwitt/downloads/test', 'rb') as f:
#     dataset = []
#     for line in f:
#         dataset.append(json.loads(line))


### 
# Eu once daily record data 
# test = Dataset(url)
# test = Dataset('https://nrt.cmems-du.eu/thredds/dodsC/dataset-wav-l4-swh-nrt-global') # once daily ESA SWH product


# len(test.variables['time'])
# test.variables['time'][-1]


In [4]:


def build_tensor_threaded(url):
    '''takes netCDF4 url and converts into a tensor'''
    to_avoid = {'time', 'lat', 'lon'}
    ds = Dataset(url)
    total_timesteps = len(ds.variables['time'])
    queue = []
    refresh = False
    logger = []
    
    dims = {}
    for key in ds.dimensions.keys():
        dims[key] = ds.dimensions[key].size
    
    variables = {}
    for num, value in enumerate(ds.variables.keys()):
        variables[num] = value
        
    c = 0
    while len(queue) > 0:    # queue of timesteps
        next_up = queue.pop(0)
        try:
            if refresh == True:
                ds  = Dataset(url)
            timestep = []
            for variable in variables:
                c += 1
                if variables[variable] in to_avoid:
                    if variables[variable] == 'time':
                        print("Starting timestep {}/{}.  SubIter {} of {}.".format(time+1, total_timesteps,c ,((len(variables) - 1) * total_timesteps)))
#                         if (time + 1) % 20 == 0:
#                             ds = Dataset(url)
#                             print('refreshing connection')
    #                     ### DEBUG ###
    #                     elif (time + 1) % 10 == 0:
    #                         return outset, variables, to_avoid
    #                     #############
                    continue
                else:
                    var_data = []
                    foc = ds[variables[variable]][time] # variable
                    timestep.append(foc.data)
            outset.append(timestep)
        except Exception as e:
            logger.append('error {}'.format(e))
            refresh = True
            ds = Dataset(url)
            queue.insert(0, next_up)
            logger.append('restarting {}'.format(next_up))
    
    outset = np.array(outset)
    outset[outset == 9.999000260554009e+20] = 0
    return outset, variables, to_avoid

In [235]:
%%time
# [1, x, x] = time
# [x, 1, x] = latitude
# [x, x, 1] = longitude
# dirpwsfc # surface primary wave direction [deg] 
# htsgwsfc # surface significant height of combined wind waves and swell [m]
# perpwsfc # surface primary wave mean period [s] 
# swdir_1  # 1 in sequence direction of swell waves [deg] 
# swdir_2  # 2 in sequence direction of swell waves [deg] 
# swdir_3  # 3 in sequence direction of swell waves [deg]
# swper_1  # 1 in sequence mean period of swell waves [s] 
# swper_2  # 2 in sequence mean period of swell waves [s]
# swper_3  # 3 in sequence mean period of swell waves [s]
# ugrdsfc  # surface u-component of wind [m/s] 
# vgrdsfc  # surface v-component of wind [m/s] 
# wdirsfc  # surface wind direction (from which blowing) [degtrue] 
# windsfc  # surface wind speed [m/s] 
# wvdirsfc # surface direction of wind waves [deg] 
# wvhgtsfc # surface significant height of wind waves [m] 
# wvpersfc # surface mean period of wind waves [s]

# Each array has 129 times
# In each time subarray theres 277 latitudes
# In each time-latitude subarray theres 1440 longitudes

# Latitudes below 45 might not have data

def build_tensor(url):
    '''takes netCDF4 url and converts into a tensor'''
    to_avoid = {'time':0, 'lat':1, 'lon':2}
    ds = Dataset(url)
    total_timesteps = len(ds.variables['time'])
    outset = []
    
    dims = {}
    for key in ds.dimensions.keys():
        dims[key] = ds.dimensions[key].size
    
    variables = {}
    for num, value in enumerate(ds.variables.keys()):
        variables[num] = value
        
    c = 0
    for time in range(dims['time']): # time
        timestep = []
        for variable in variables:
            c += 1
            if variables[variable] in to_avoid:
                if variables[variable] == 'time':
                    print("Starting timestep {}/{}.  SubIter {} of {}.".format(time+1, total_timesteps,c ,((len(variables) - 1) * total_timesteps)))
                    if (time + 1) % 20 == 0:
                        ds = Dataset(url)
                        print('refreshing connection')
#                     ### DEBUG ###
#                     elif (time + 1) % 10 == 0:
#                         return outset, variables, to_avoid
#                     #############
                continue
            else:
                var_data = []
                foc = ds[variables[variable]][time] # variable
                timestep.append(foc.data)
        outset.append(timestep)
    
    outset = np.array(outset)
    outset[outset == 9.999000260554009e+20] = 0
    return outset, variables, to_avoid
 
# Data collection keeps erroring out, need a solution to this.  Understand opendap and how to grab as much data
# Ask quickly as possible, multithreading, chunking, etc
data, variables, avoided = build_tensor(url)

Starting timestep 1/129.  SubIter 1 of 2709.
Starting timestep 2/129.  SubIter 23 of 2709.
Starting timestep 3/129.  SubIter 45 of 2709.
Starting timestep 4/129.  SubIter 67 of 2709.
Starting timestep 5/129.  SubIter 89 of 2709.
Starting timestep 6/129.  SubIter 111 of 2709.
Starting timestep 7/129.  SubIter 133 of 2709.
Starting timestep 8/129.  SubIter 155 of 2709.
Starting timestep 9/129.  SubIter 177 of 2709.
Starting timestep 10/129.  SubIter 199 of 2709.
Starting timestep 11/129.  SubIter 221 of 2709.
Starting timestep 12/129.  SubIter 243 of 2709.
Starting timestep 13/129.  SubIter 265 of 2709.
Starting timestep 14/129.  SubIter 287 of 2709.
Starting timestep 15/129.  SubIter 309 of 2709.
Starting timestep 16/129.  SubIter 331 of 2709.
Starting timestep 17/129.  SubIter 353 of 2709.
Starting timestep 18/129.  SubIter 375 of 2709.
Starting timestep 19/129.  SubIter 397 of 2709.
Starting timestep 20/129.  SubIter 419 of 2709.
refreshing connection
Starting timestep 21/129.  SubIte

In [ ]:
### Threaded test ###
data_2 = build_tensor_threaded(url)

In [237]:
sys.getsizeof(data)  * 1e-6

3910.61968

In [ ]:
def shrink_tensor_alt(tensor, variable_order):
    tensor = np.array(tensor)
    starting_size = sys.getsizeof(tensor)
    shape = tensor.shape
    out_tensor = np.array(shape)
    finish_flag = False
    errors = []
#     temp_hash = []
    test_hash = []
    for timestep in range(shape[0]):
        var_hash = {}
        for variable in range(shape[1]):
            lat_arr = []
            if variable == shape[1] - 1:
                finish_flag = True
            for latitude in range(shape[2]):
                latitude = str(latitude)
                if var_hash.get(latitude) == None:
                    var_hash[latitude] = {}
                lon_arr = []
                for longitude in range(shape[3]):
                    longitude = str(longitude)
                    if var_hash[latitude].get(longitude) == None:
                        var_hash[latitude][longitude] = {}
                    if var_hash.get(latitude, {}).get(longitude, {}).get(variable_order[variable]):
                        errors.append("error var:{} lat:{} long:{} time:{}".format(variable_order[variable]))
                    else:
                        var_hash[latitude][longitude][variable_order[variable]] = tensor[timestep][variable][int(latitude)][int(longitude)]
                        if finish_flag:
                            lon_arr.append(np.array(list(var_hash[latitude][longitude].values()), dtype=np.float16))
                                           
                if finish_flag:
                    lat_arr.append(np.array(lon_arr, dtype=np.float16))
        if finish_flag:
            test_hash.append(np.array(lat_arr, dtype=np.float16))
            print("finished {} of {}".format(timestamp, shape[0]))
#         temp_hash.append(var_hash)
        # Once verified that all data is there, need to convert to array dict
        
        
        
    return test_hash, errors
                        

In [11]:
sys.getsizeof(80)

28

In [4]:
### Function to decrease size of tensor, ~4 gbs per file currently
# Do triple loop to go through each orientation, grab each individual data point, organize it
# time > variable > lat > lon > data
# time > lat > lon > [var1, var2, var3] ## NEW

# To shrink the tensor, the options are either to learn topology (grab whole neighborhoods )

# variable_order = list(set(variables.values()) - set(avoided))
variable_order = list(x.keys())
external = ''
def shrink_tensor(tensor, variable_order):
    tensor = np.array(tensor)
    starting_size = sys.getsizeof(tensor)
    shape = tensor.shape
    finish_flag = False
    errors = []
    test_hash = []
    for timestep in range(shape[0]):
        var_hash = {}
        for variable in range(shape[1]):
            lat_arr = []
            if variable == shape[1] - 1:
                finish_flag = True
            for latitude in range(shape[2]):
                latitude = str(latitude)
                if var_hash.get(latitude) == None:
                    var_hash[latitude] = {}
                lon_arr = []
                for longitude in range(shape[3]):
                    longitude = str(longitude)
                    if var_hash[latitude].get(longitude) == None:
                        var_hash[latitude][longitude] = {}
                    if var_hash.get(latitude, {}).get(longitude, {}).get(variable_order[variable]):
                        errors.append("error var:{} lat:{} long:{} time:{}".format(variable_order[variable]))
                    else:
                        var_hash[latitude][longitude][variable_order[variable]] = tensor[timestep][variable][int(latitude)][int(longitude)]
                        if finish_flag:
                            lon_arr.append(list(var_hash[latitude][longitude].values()))
                                           
                if finish_flag:
                    lat_arr.append(np.array(lon_arr, dtype=np.float16))
        if finish_flag:
            print(sys.getsizeof(test_hash) ** 1e-6)
            test_hash.append(np.array(lat_arr, dtype=np.float16))
            print("finished {} of {}".format(timestep, shape[0]))
        
        
        
    return test_hash, errors
                        

In [14]:
# del data
data = load_file('nparray', 'pull_one.npy')



In [46]:
data[0][10][100][0]
# data[0][10][100][0]

9.65

In [39]:
a[0].shape

(277, 1440, 19)

In [48]:
a[0][100][0][10]

9.65

In [7]:
%%time
data = load_file('nparray', 'pull_one.npy')
f = np.array(data)
del data

ftest, errors = shrink_tensor(f, variable_order)

1.0000040253597924
finished 0 of 129
1.0000044773468377
finished 1 of 129
1.0000044773468377
finished 2 of 129
1.0000044773468377
finished 3 of 129
1.0000044773468377
finished 4 of 129
1.0000047875032028
finished 5 of 129
1.0000047875032028
finished 6 of 129
1.0000047875032028
finished 7 of 129
1.0000047875032028
finished 8 of 129
1.0000052149493555
finished 9 of 129
1.0000052149493555
finished 10 of 129
1.0000052149493555
finished 11 of 129
1.0000052149493555
finished 12 of 129
1.0000052149493555
finished 13 of 129
1.0000052149493555
finished 14 of 129
1.0000052149493555
finished 15 of 129
1.0000052149493555
finished 16 of 129
1.0000055134439452
finished 17 of 129
1.0000055134439452
finished 18 of 129
1.0000055134439452
finished 19 of 129
1.0000055134439452
finished 20 of 129
1.0000055134439452
finished 21 of 129
1.0000055134439452
finished 22 of 129
1.0000055134439452
finished 23 of 129
1.0000055134439452
finished 24 of 129
1.000005743019679
finished 25 of 129
1.000005743019679
finis

In [10]:
a = np.array(ftest)

In [6]:
%%time
def save_file(file, filetype, filename='test'):
    root_path = os.path.expanduser('~/downloads')
    if filetype == 'nparray':
#         np.savetxt(root_path+filename, file)
        np.save(root_path+'/'+filename+'.npy', data)
    elif filetype == 'netcdf':
        pass
    
def load_file(filetype, filename):
    if filetype == 'nparray':
        file = np.load(os.path.expanduser('~/downloads') +'/'+ filename, allow_pickle=True)
    
    return file
# save_file(data, filetype='nparray', filename='pull_one')
# data = load_file('nparray', 'pull_one.npy')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 4.77 µs


In [3]:
x = {'perpwsfc': 277.29,
 'swdir_2': 5.19,
 'swper_1': 11.759999,
 'ugrdsfc': 9.999e+20,
 'swper_3': 9.999e+20,
 'htsgwsfc': 9.999e+20,
 'wdirsfc': 9.999e+20,
 'swell_2': 9.999e+20,
 'wvdirsfc': 9.999e+20,
 'dirpwsfc': 9.999e+20,
 'vgrdsfc': 9.999e+20,
 'wvpersfc': 9.999e+20,
 'swell_1': 10.44,
 'swdir_1': 14.12,
 'swdir_3': 216.47,
 'swper_2': 17.56,
 'swell_3': 274.38,
 'windsfc': 5.19,
 'wvhgtsfc': 11.759999}



In [232]:
sys.getsizeof(f) * 1e-6

272.83408

In [233]:
sys.getsizeof(np.array(ftest)) * 1e-6

136.41711999999998

In [243]:
test[0]['100']['400']

{'perpwsfc': 277.29,
 'swdir_2': 5.19,
 'swper_1': 11.759999,
 'ugrdsfc': 9.999e+20,
 'swper_3': 9.999e+20,
 'htsgwsfc': 9.999e+20,
 'wdirsfc': 9.999e+20,
 'swell_2': 9.999e+20,
 'wvdirsfc': 9.999e+20,
 'dirpwsfc': 9.999e+20,
 'vgrdsfc': 9.999e+20,
 'wvpersfc': 9.999e+20,
 'swell_1': 10.44,
 'swdir_1': 14.12,
 'swdir_3': 216.47,
 'swper_2': 17.56,
 'swell_3': 274.38,
 'windsfc': 5.19,
 'wvhgtsfc': 11.759999}

In [20]:
len(data[0])#[-1][45][70]

19

In [151]:
a = np.array([1.1,2.1,3.1],dtype=float)
b = np.array([1.2,2.2,3.2], dtype=float)
c = np.array([a,b], dtype=float)
c
# np.array(, a, dtype=float)

array([[1.1, 2.1, 3.1],
       [1.2, 2.2, 3.2]])

In [21]:
(sys.getsizeof(f) * 1e-6)

3910.61968

In [69]:
sys.getsizeof(test)

56

In [22]:
a = np.array([1,2,3])
b = np.array([4,5,6])
print(np.array([a,b]).shape)


a = [1,2,3]
b = [4,5,6]
print(np.array([a,b]).shape)

a = [[[1],[2],[3]]]
b = [[[1],[2],[3]]]
print(np.array([a,b]).shape)
np.array([a,b], dtype=float)

(2, 3)
(2, 3)
(2, 1, 3, 1)


array([[[[1.],
         [2.],
         [3.]]],


       [[[1.],
         [2.],
         [3.]]]])

In [133]:
np.array(data)[0][0][0]

array([9.999e+20, 9.999e+20, 9.999e+20, ..., 9.999e+20, 9.999e+20,
       9.999e+20], dtype=float32)

In [ ]:
from app.Pipeline.sat_dat import Satellite

In [ ]:
dumb = Satellite() # failed on timestep 79, 82
second_test = dumb.build_tensor(url)

In [ ]:
# Convert interesting data (wavedir, waveheight) to lat/long grid, convert nulls to 0s